# Imports

Please use the provided `hds_stoch` environment for this work.  

In [1]:
import numpy as np
import pandas as pd
import itertools
import math
import matplotlib.pyplot as plt
import simpy
from joblib import Parallel, delayed
from treat_sim.distributions import Exponential, Lognormal

# Utility functions

In [2]:
def trace(msg):
    '''
    Utility function for printing simulation
    set the TRACE constant to FALSE to 
    turn tracing off.
    
    Params:
    -------
    msg: str
        string to print to screen.
    '''
    if TRACE:
        print(msg)

# Model parameters

The constants below provides hard coded data representing the base case or 'as-is' state of the minor injury unit.   

In [69]:
# These are the parameters for a base case model run.

# run length in days
RUN_LENGTH = 365

# default № of reps for multiple reps run
DEFAULT_N_REPS = 5

# default random number SET
DEFAULT_RNG_SET = None
N_STREAMS = 10

# Turn off tracing
TRACE = False

# resource counts
N_BEDS = 10

# time between arrivals in minutes (exponential)
# for acute stroke, TIA and neuro respectively
MEAN_IATs = [1.2, 9.5, 3.5]

# treatment (lognormal)
# for acute stroke, TIA and neuro respectively
TREAT_MEANs = [7.4, 1.8, 2.0]
TREAT_STDs = [8.5, 2.3, 2.5]

# Scenario class
In the cell below the parameters you will find a `Scenario` class.  This makes use of the default parameters to set up the base case scenario.  Remember that its good practice to pass all of your parameters to your simulation model in a **container**.  A class is a flexible way to achieve this aim.

In [4]:
class Scenario:
    '''
    Parameter container class for minor injury unit model.
    '''
    def __init__(self, random_number_set=DEFAULT_RNG_SET):
        '''
        The init method sets up our defaults. 
        
        Params:
        -------
        
        name - str or None
            optional name for scenario
        '''
        
        # warm-up
        self.warm_up = 0.0
        
        # either hard-coded or obtained from streamlit
        self.iat_means = MEAN_IATs
        self.treat_means = TREAT_MEANs
        self.treat_stds = TREAT_STDs

        # sampling
        self.random_number_set = random_number_set
        self.init_sampling()
        
        # number of beds
        self.beds = N_BEDS
        
        
    def set_random_no_set(self, random_number_set):
        '''
        Controls the random sampling 

        Parameters:
        ----------
        random_number_set: int
            Used to control the set of psuedo random numbers
            used by the distributions in the simulation.
        '''
        self.random_number_set = random_number_set
        self.init_sampling()
        
        
    def init_sampling(self):
        '''
        Create the distributions used by the model and initialise 
        the random seeds of each.
        '''
        # create random number streams
        rng_streams = np.random.default_rng(self.random_number_set)
        self.seeds = rng_streams.integers(0, 999999999, size=N_STREAMS)
        
        
        # inter-arrival distributions samples
        self.arrival_dist_samples = {
            'stroke': Exponential(self.iat_means[0], random_seed=self.seeds[0]),
            'tia': Exponential(self.iat_means[1], random_seed=self.seeds[1]),
            'neuro': Exponential(self.iat_means[2], random_seed=self.seeds[2])
        }
        
        # treatment distributions samples
        self.treatment_dist_samples = {
            'stroke': Lognormal(self.treat_means[0], self.treat_stds[0], random_seed=self.seeds[3]),
            'tia': Lognormal(self.treat_means[1], self.treat_stds[1], random_seed=self.seeds[4]),
            'neuro': Lognormal(self.treat_means[2], self.treat_stds[2], random_seed=self.seeds[5])
        }
            
        

# Model building

In [55]:
class Patient:
    '''
    Patient in the minor ED process
    '''
    def __init__(self, identifier, patient_type, env, args):
        '''
        Constructor method
        
        Params:
        -----
        identifier: int
            a numeric identifier for the patient.
            
        env: simpy.Environment
            the simulation environment
            
        args: Scenario
            The input data for the scenario
        '''
        # patient id and environment
        self.identifier = identifier
        self.env = env
        
        self.patient_type = patient_type

        # treatment parameters
        self.beds = args.beds
        self.treatment_dist_samples = args.treatment_dist_samples
                
        # individual patient metrics
        self.queue_time = 0.0
        self.treat_time = 0.0
        
    def get_treatment_dist_sample(self):

        # sample for patient pathway
        if self.patient_type == 'stroke':
            self.treat_time = self.treatment_dist_samples['stroke'].sample()
        elif self.patient_type == 'tia':
            self.treat_time = self.treatment_dist_samples['tia'].sample()
        elif self.patient_type == 'neuro':
            self.treat_time = self.treatment_dist_samples['neuro'].sample()  
        else:
            trace('???')
                
        activity_duration = self.treat_time
            
        return activity_duration
        
    
    def treatment(self):

        # record the time that patient entered the system
        arrival_time = self.env.now
     
        # get a bed
        with self.beds.request() as req:
            yield req
            
            # record time to first being seen by a doctor
            self.queue_time = self.env.now - arrival_time

            trace(f'Patient № {self.identifier} started treatment at {self.env.now:.3f};'
                      + f' queue time was {self.queue_time:.3f}')            

            # treatment delay
            yield self.env.timeout(self.get_treatment_dist_sample())

In [67]:
class ASU:  
    '''
    Model of an ASU
    '''
    def __init__(self, args):
        '''
        Contructor
        
        Params:
        -------
        env: simpy.Environment
        
        args: Scenario
            container class for simulation model inputs.
        '''
        self.env = simpy.Environment()
        self.args = args 
        self.init_model_resources(args)
        self.patients = []
        self.patient_count = 0
        self.stroke_count = 0
        self.tia_count = 0
        self.neuro_count = 0
        
        
    def init_model_resources(self, args):
        '''
        Setup the simpy resource objects
        
        Params:
        ------
        args - Scenario
            Simulation Parameter Container
        '''

        args.beds = simpy.Resource(self.env, 
                                   capacity=args.beds)
        
        
    def run(self, results_collection_period = RUN_LENGTH,
            warm_up = 0):
        '''
        Conduct a single run of the model in its current 
        configuration

        run length = results_collection_period + warm_up

        Parameters:
        ----------
        results_collection_period, float, optional
            default = RUN_LENGTH

        warm_up, float, optional (default=0)
            length of initial transient period to truncate
            from results.

        Returns:
        --------
            None

        '''
        # setup the arrival processes
        self.env.process(self.arrivals_generator('stroke'))
        self.env.process(self.arrivals_generator('tia'))
        self.env.process(self.arrivals_generator('neuro'))
                
        # run
        self.env.run(until=results_collection_period+warm_up)
        
        
    def get_arrival_dist_sample(self):
        
        if self.patient_type == 'stroke':
            inter_arrival_time = self.args.arrival_dist_samples['stroke'].sample()
        elif self.patient_type == 'tia':
            inter_arrival_time = self.args.arrival_dist_samples['tia'].sample()
        elif self.patient_type == 'neuro':
            inter_arrival_time = self.args.arrival_dist_samples['neuro'].sample()
        else:
            trace('???')
        
        return inter_arrival_time
            
        
    def arrivals_generator(self, patient_type):
        self.args.init_sampling()
            
        while True:
                
                self.patient_type = patient_type
            
                if self.patient_type == 'stroke':
                    self.stroke_count += 1
                elif self.patient_type == 'tia':
                    self.tia_count += 1
                elif self.patient_type == 'neuro':
                    self.neuro_count += 1    
                else:
                    trace('???')
                
                yield self.env.timeout(self.get_arrival_dist_sample())
                
                self.patient_count += 1

                trace(f'Patient № {self.patient_count} ({patient_type}) arrives at: {self.env.now:.3f}')
                
                # create a new patient and pass in env and args
                new_patient = Patient(self.patient_count, patient_type, self.env, self.args)                

                # init the minor injury process for this patient
                self.env.process(new_patient.treatment())                 
            
                # keep a record of the patient for results calculation
                self.patients.append(new_patient)                
                
                
                
    def run_summary_frame(self):
        
        '''
        Utility function for final metrics calculation.
        
        Returns df containining
        total pts count
        count of each of pts types
        bed utilisation
        mean waiting time of bottom 90% of pts
        % of pts admitted in <4 hours
        '''
        # create nparray of all queue times, convert to hours
        patients_queue_times = np.array([patient.queue_time * 24 for patient in self.patients])
        
        #sort array in descending order, drops top 10% of values, calculate the mean 
        queue_bottom90 = np.mean(np.sort(patients_queue_times)[len(self.patients) // 10:])
        
        
        # calculate proportion of patient with queue time less than 4 hrs
        percent_4_less = (sum(patients_queue_times <= 4)/len(self.patients))*100
    
        #bed utilisation = sum(call durations) / (run length X no. BEDS)
        util = np.array([patient.treat_time 
                     for patient in self.patients]).sum() / (RUN_LENGTH * N_BEDS)
        
        average_treat_time = np.array([patient.treat_time 
                     for patient in self.patients]).sum() / self.patient_count
        

        df = pd.DataFrame({'1':{'1b Stroke Patient Arrivals':self.stroke_count,
                                '2 Bottom 90% Mean Treatment Waiting Time (hrs)': queue_bottom90, 
                                '4 Bed Utilisation (%)': util*100,
                                '1a Total Patient Arrivals':self.patient_count,
                                '1d Neuro Patient Arrivals':self.neuro_count,
                                '1c TIA Patient Arrivals':self.tia_count,
                                '5 Mean Total Time in Unit per patient(hrs)':average_treat_time,
                                '3 Patients Admitted to Unit within 4 hrs of arrival(%)': percent_4_less}})
        
        
        df = df.T
        df.index.name = 'rep'
        return df

# Functions for single and multiple runs

In [70]:
def single_run(scenario, 
               rc_period = RUN_LENGTH, 
               warm_up = 0,
               random_no_set = DEFAULT_RNG_SET):
    '''
    Perform a single run of the model and return the results
    
    Parameters:
    -----------
    
    scenario: Scenario object
        The scenario/paramaters to run
        
    rc_period: int
        The length of the simulation run that collects results
        
    warm_up: int, optional (default=0)
        warm-up period in the model.  The model will not collect any results
        before the warm-up period is reached.  
        
    random_no_set: int or None, optional (default=1)
        Controls the set of random seeds used by the stochastic parts of the 
        model.  Set to different ints to get different results.  Set to None
        for a random set of seeds.
        
    Returns:
    --------
        pandas.DataFrame:
        results from single run.
    '''  
        
    # set random number set - this controls sampling for the run.
    scenario.set_random_no_set(random_no_set)
    
    # create the model
    model = ASU(scenario)

    model.run(results_collection_period = rc_period, warm_up = warm_up)
    
    # run the model
    results_summary= model.run_summary_frame()
    
    return results_summary

In [8]:
def multiple_replications(scenario, 
                          rc_period=RUN_LENGTH,
                          warm_up=0,
                          n_reps=DEFAULT_N_REPS, 
                          n_jobs=-1):
    '''
    Perform multiple replications of the model.
    
    Params:
    ------
    scenario: Scenario
        Parameters/arguments to configurethe model
    
    rc_period: float, optional (default=DEFAULT_RESULTS_COLLECTION_PERIOD)
        results collection period.  
        the number of minutes to run the model beyond warm up
        to collect results
    
    warm_up: float, optional (default=0)
        initial transient period.  no results are collected in this period

    n_reps: int, optional (default=DEFAULT_N_REPS)
        Number of independent replications to run.

    n_jobs, int, optional (default=-1)
        No. replications to run in parallel.
        
    Returns:
    --------
    List
    '''    
    res = Parallel(n_jobs=n_jobs)(delayed(single_run)(scenario, 
                                                      rc_period, 
                                                      warm_up,
                                                      random_no_set=rep) 
                                  for rep in range(n_reps))


    # format and return results in a dataframe
    df_results = pd.concat(res)
    df_results.index = np.arange(1, len(df_results)+1)
    df_results.index.name = 'rep'
    return df_results


# Scripts to run the model

In [71]:
# SINGLE RUN

# base case scenario with default parameters
default_args = Scenario()

print('Running simulation ...', end = ' => ')

results = single_run(default_args)

print('simulation complete.')

results

Running simulation ... => simulation complete.


,1a Total Patient Arrivals,1b Stroke Patient Arrivals,1c TIA Patient Arrivals,1d Neuro Patient Arrivals,2 Bottom 90% Mean Treatment Waiting Time (hrs),3 Patients Admitted to Unit within 4 hrs of arrival(%),4 Bed Utilisation (%),5 Mean Total Time in Unit per patient(hrs)
rep,,,,,,,,
1,427.0,282.0,44.0,104.0,3.591513,88.290398,65.828373,5.627016


In [72]:
# MULTIPLE RUN

# default scenario
args = Scenario()

# run multiple replications.
# by default it runs 5 replications.
print('Running multiple replications', end=' => ')
results  = multiple_replications(args, n_reps=10)
print('done.\n')

# show reps rounded to 2dp
results.round(2)

Running multiple replications => done.



,1a Total Patient Arrivals,1b Stroke Patient Arrivals,1c TIA Patient Arrivals,1d Neuro Patient Arrivals,2 Bottom 90% Mean Treatment Waiting Time (hrs),3 Patients Admitted to Unit within 4 hrs of arrival(%),4 Bed Utilisation (%),5 Mean Total Time in Unit per patient(hrs)
rep,,,,,,,,
1,454.0,325.0,43.0,89.0,12.63,77.75,75.93,6.10
2,445.0,309.0,44.0,95.0,3.54,87.87,66.09,5.42
3,477.0,336.0,27.0,117.0,11.36,80.29,74.16,5.68
4,429.0,294.0,36.0,102.0,7.62,84.85,61.75,5.25
5,466.0,319.0,35.0,115.0,6.09,88.63,66.94,5.24
6,421.0,281.0,50.0,93.0,5.05,86.94,61.66,5.35
7,447.0,319.0,32.0,99.0,20.77,69.35,76.59,6.25
8,446.0,299.0,30.0,120.0,13.48,76.68,73.83,6.04
9,430.0,314.0,18.0,101.0,32.69,68.60,74.67,6.34


In [11]:
results.mean()

1a Total Patient Arrivals                                 433.800000
1b Stroke Patient Arrivals                                 52.600000
1c TIA Patient Arrivals                                   201.100000
1d Neuro Patient Arrivals                                 183.100000
2 Bottom 90% Mean Treatment Waiting Time (hrs)              0.001967
3 Patients Admitted to Unit within 4 hrs of arrival(%)     99.976798
4 Bed Utilisation (%)                                      30.683554
5 Mean Total Time in Unit per patient(hrs)                  2.592670
dtype: float64